In [1]:
import numpy as np
from skimage.transform import resize
from skimage import color
import matplotlib.pyplot as plt
import IPython
from neutorch.dataset.border_mask import create_border_mask

%load_ext autoreload
%autoreload 2
%matplotlib inline

def scale_data(vol, seg, size=180):
    length = vol.shape[0]
    vol = resize(vol, (length, size, size))
    seg = resize(seg, (length, size, size), order=0, preserve_range=True, anti_aliasing=False)

    return (vol, seg)
    
def label_data(vol, seg):
    length = vol.shape[0]
    size = vol.shape[1]
    # reshape for labeling
    seg = np.reshape(seg, (size, length*size) )
    vol = np.reshape(vol, (size, length*size) )
    # label
    labeled = color.label2rgb(seg, vol, alpha=0.1, bg_label=-1)
    # shape back
    labeled = np.reshape(labeled, (length, size, size, 3) )

    return labeled

def vis_volume(volume, fig_size=6.5):
    length = volume.shape[0]
    
    # set up
    fig = plt.figure()
    fig.set_size_inches(fig_size, fig_size)
    axes = fig.add_subplot()
    hfig = display(fig, display_id=True)

    # display
    for i in range(length):
        axes.imshow(volume[i], cmap='gray', interpolation='none', filternorm=False, resample=False)
        fig.canvas.draw()
        hfig.update(fig)
        if i != length-1:
            plt.cla()
    # clean up
    IPython.display.clear_output()

def view_patch(patch,i=0):
    plt.rcParams['image.interpolation'] = 'nearest'
    fig, axarr = plt.subplots(3,4)
    fig.set_size_inches(16, 10)
    axarr[0,0].imshow(patch.image[0][i], cmap='gray')
    axarr[0,1].imshow(patch.label[0][i], cmap='prism')
    axarr[0,2].imshow(label_data(patch.image[0],patch.label[0])[i])
    axarr[1,0].imshow(patch.affinity[0][i])
    axarr[1,1].imshow(patch.affinity[1][i])
    axarr[1,2].imshow(patch.affinity[2][i])
    axarr[2,0].imshow(patch.get_lsd_channel(0)[i,:,:,0])
    axarr[2,1].imshow(patch.get_lsd_channel(1)[i,:,:,0])
    axarr[2,2].imshow(patch.get_lsd_channel(2)[i,:,:,0])
    axarr[2,3].imshow(patch.get_lsd_channel(3)[i], cmap='gray')
    plt.show()

def view_affinity(affinity, i=0, fig_size=2):
    plt.rcParams['image.interpolation'] = 'nearest'
    num_aff = affinity.shape[0]
    fig, axarr = plt.subplots(1,num_aff)
    fig.set_size_inches(fig_size*num_aff, fig_size)
    for a in range(num_aff):
        axarr[a].imshow(affinity[a][i], cmap='gray')
        axarr[a].imshow(affinity[a][i], cmap='gray')
        axarr[a].imshow(affinity[a][i], cmap='gray')
    plt.show()

def view_segmentation(seg,i=0, fig_size=10, border=0):
    if border > 0:
        masked_label = np.zeros(seg.shape, dtype=np.uint64)
        create_border_mask(seg, masked_label, border, 0)
        seg = masked_label

    fig = plt.figure()
    fig.set_size_inches(fig_size, fig_size)
    colored = color.label2rgb(seg[i], alpha=1, bg_label=0)
    plt.imshow(colored)
    plt.show()

In [19]:
from neutorch.dataset.utils import from_h5
affinityAp = from_h5('../../RSUnetBIG/aff_sample_A+_pad.h5', dataset_path='affinity')
affinityAp = affinityAp[:, 37:37+125, 911:911+1250, 911:911+1250]


In [21]:
import cc3d
import numpy as np

counts = []
for i in range(affinityAp.shape[1]):
    aff_sl = affinityAp[:,i,:,:].astype(int)
    labels_out = cc3d.connected_components(aff_sl,connectivity=6) 
    counts.append(len(np.unique(labels_out)))

In [23]:
arg_mins = np.argsort(counts)
print(arg_mins)

[ 80  79 108 109  33   0  81   1  34  32 110 107  78 106 105 111  59  57
  77  67 103  95  58  97  51  71 104  98  60  70 101  69 100  17  96  15
  76  31  27  68  63  16  61  62  41  66  42  19  38 102  73  29  56  65
  99  64  39  24 112  72  35  30  28  37  20  40  25  23  74  75  55  26
  21   2  18  94  82  22  36  14  52  43  13 113  92  93   3  44  83  91
 122  12  45   4 116 114  11 123  54 115  89  88  48   5  85  50  84  49
  86  46 121  53  90 124   9   8   6  47  87   7  10 117 119 118 120]


In [28]:
from neutorch.dataset.utils import from_h5

segA = from_h5(f'../../RSUnet2_800000_run/seg_sample_A+_pad.hdf', dataset_path='volumes/labels/neuron_ids')
segB = from_h5(f'../../RSUnet2_800000_run/seg_sample_B+_pad.hdf', dataset_path='volumes/labels/neuron_ids')
segC = from_h5(f'../../RSUnet2_800000_run/seg_sample_C+_pad.hdf', dataset_path='volumes/labels/neuron_ids')


In [29]:
imgA = from_h5(f'../../data/sample_A+.hdf', dataset_path='volumes/raw')
imgB = from_h5(f'../../data/sample_B+.hdf', dataset_path='volumes/raw')
imgC = from_h5(f'../../data/sample_C+.hdf', dataset_path='volumes/raw')

In [7]:
print(segA.shape)
print(segB.shape)
print(segC.shape)

(200, 3072, 3072)
(200, 3072, 3072)
(200, 3072, 3072)


In [26]:
import math
def grid_vol(vol, sz=100):
    num_sl = vol.shape[0]
    r = math.ceil(math.sqrt(num_sl))
    fig, axarr = plt.subplots(r,r)
    fig.set_size_inches(sz, sz)
    for i in range(r):
        for j in range(r):
            cur = (i*r)+j
            if cur < num_sl:
                axarr[i,j].imshow(vol[cur], cmap='gray')
                axarr[i,j].set_title(cur)
    plt.axis('off')            
    plt.show()

In [30]:
segA = color.label2rgb(segA, alpha=1, bg_label=0)

In [32]:
segB = color.label2rgb(segB, alpha=1, bg_label=0)

In [34]:
segC = color.label2rgb(segC, alpha=1, bg_label=0)

In [21]:
imgA = from_h5(f'../../data/sample_A+_pad.hdf', dataset_path='volumes/raw')